# Message table/translations

`Mech3Msg.dll` contains a message table:

```console
$ rabin2 -U Mech3Msg.dll
Resource 0
  name: 1
  timestamp: Thu Jan  1 00:00:00 1970
  vaddr: 0x1000e060
  size: 64.9K
  type: MESSAGETABLE
  language: LANG_ENGLISH
```

The German version predictably has the language `LANG_GERMAN`. This isn't an uncommon way of handling localisation. In fact Microsoft describes a similar approach to ["localizing message strings"](https://docs.microsoft.com/en-us/windows/win32/wes/localizing-message-strings). With some help from structures in `Winnt.h` - [MESSAGE_RESOURCE_DATA](https://docs.microsoft.com/en-us/windows/win32/api/winnt/ns-winnt-message_resource_data), [MESSAGE_RESOURCE_BLOCK](https://docs.microsoft.com/en-us/windows/win32/api/winnt/ns-winnt-message_resource_block), and [MESSAGE_RESOURCE_ENTRY](https://docs.microsoft.com/en-us/windows/win32/api/winnt/ns-winnt-message_resource_entry) - it's trivial to read the message table on all platforms, without Windows APIs (I believe internally the executable uses `FormatMessage`).

What is uncommon is the export, localisation string DLLs usually have none:

```console
$ rabin2 -E Mech3Msg.dll
[Exports]
000 0x00000b20 0x10001720 GLOBAL   FUNC    0 Mech3Msg.dll_ZLocGetID
```

There's also quite a lot of code in the DLL initialisation, and a lot of strings - more than double what's in the message table. A lot of these strings have a common prefix:


```console
$ strings Mech3Msg.dll | wc -l
    3763
$ strings Mech3Msg.dll | grep -F "MSG_"
MSG_NOTEXTURE_MEMORY
MSG_REALLOC_SALVAGE
MSG_NO_END
MSG_NOT_ALONE
MSG_ALL_ALONE
[...]
```

After some reverse engineering, or simply trying some different arguments, it becomes apparent that when `ZLocGetID` is passed one of those strings like `MSG_NOTEXTURE_MEMORY`, it returns an unsigned 32-bit integer which corresponds to the message ID in the table. In Python - but only on Windows - this can be done as follows:

In [1]:
try:
    import ctypes
    import ctypes.wintypes
except ValueError:
    print("only works on Windows")
else:
    lib = ctypes.CDLL("Mech3Msg.dll")
    ZLocGetID = lib.ZLocGetID
    ZLocGetID.argtypes = [ctypes.c_char_p]
    ZLocGetID.restype = ctypes.c_int32

    # message_id = ZLocGetID(message_name)

only works on Windows


All these strings seem to be stored in a rather large `.data` section (the `.rsrc` section [where resources are stored](https://docs.microsoft.com/en-us/windows/win32/debug/pe-format#the-rsrc-section), in this case just the message table):
```
$ rabin2 -S Mech3Msg.dll
[Sections]
00 0x00000400  3072 0x10001000  4096 -r-x .text
01 0x00001000   512 0x10002000  4096 -r-- .rdata
02 0x00001200 45056 0x10003000 45056 -rw- .data
03 0x0000c200 67072 0x1000e000 69632 -r-- .rsrc
04 0x0001c800  3584 0x1001f000  4096 -r-- .reloc
```

The messages can be extracted in a hacky way (they are padded to 4 bytes with zeros which helps). Luckily, there is also a table in the `.data` section that encoded the message name address, and the message table ID (in reversed order). This is pretty reliable, although detecting the end of the table is a bit janky. There is an unknown value. For zeroed out entries, it's 0. For all others, it seems to be 4096.

All of this is to say it's now possible to extract the message names and values. This code isn't very interesting as part of the write-up, as it's largely parsing the portable executable (PE) format and implementing functionality to read Windows message tables. 

Bonus facts:

1. Not all messages have corresponding values in the message table - it was probably easier to leave them in, knowing they're unused than recreate this data.
2. Some messages are zeroed out by the patch, for example `MSG_GAME_NAME_DEBUG_VER`. I think that's rather interesting!

## Next up

[`reader*.zbd` archives/game data](10-reader.ipynb)